In [67]:
import numpy as np
import pandas as pd
import h5py

In [9]:
%%time
# Download TCGA+TARGET+GTEX from Xena loading directly as float32 vs. pandas float64 default
# https://toil.xenahubs.net/download/TcgaTargetGtex_rsem_gene_tpm
converters = defaultdict(str)
converters["sample"] = str
X = pd.read_csv("/data/references/xena/TcgaTargetGtex_rsem_gene_tpm",
                         sep="\t", index_col=0, converters=converters, dtype=np.float32)
print("Type: {} Total Size Bytes: {}".format(expression.ix[1,1], expression.values.nbytes))
print("Shape: {}".format(expression.shape))

# Save as hdf5 via pandas
X.to_hdf("/data/scratch/rcurrie/xena/TcgaTargetGtex_rsem_gene_tpm.hd5", "expression", mode="w", format="fixed")

Type: -9.965800285339355 Total Size Bytes: 4660765920
Shape: (60498, 19260)
CPU times: user 16min 45s, sys: 12.2 s, total: 16min 57s
Wall time: 16min 57s


In [29]:
# Read back hdf using pandas
X = pd.read_hdf("/data/scratch/rcurrie/xena/TcgaTargetGtex_rsem_gene_tpm.hd5", "expression")
X.head()

,GTEX-UTHO-1226-SM-3GAEE,GTEX-146FH-1726-SM-5QGQ2,GTEX-QDT8-0126-SM-48TZ1,GTEX-QCQG-1326-SM-48U24,GTEX-WZTO-2926-SM-3NM9I,GTEX-12WSB-0126-SM-59HJN,GTEX-11VI4-0626-SM-5EQLO,GTEX-T5JC-0526-SM-32PM7,GTEX-RU1J-0426-SM-46MUK,GTEX-1212Z-0226-SM-59HLF,...,TCGA-AB-2965-03,TCGA-AB-2936-03,TCGA-AB-2839-03,TCGA-AB-2879-03,TCGA-AB-2886-03,TCGA-AB-2901-03,TCGA-AB-2862-03,TCGA-AB-2956-03,TCGA-AB-2987-03,TCGA-AB-2868-03
sample,,,,,,,,,,,,,,,,,,,,,
ENSG00000242268.2,-9.9658,-9.9658,-9.9658,-9.9658,-1.1488,-9.9658,-9.9658,-1.1172,-9.9658,-3.8160,...,-2.5479,1.7053,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658
ENSG00000259041.1,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658
ENSG00000270112.3,-1.2828,-9.9658,-6.5064,-9.9658,0.7146,-6.5064,-5.5735,-6.5064,-6.5064,-9.9658,...,-9.9658,-3.1714,-4.2934,-4.0350,-2.4659,-3.0469,-2.1779,-9.9658,-9.9658,-6.5064
ENSG00000167578.16,3.3717,5.0215,5.3173,5.6834,6.1991,4.5485,4.5491,4.2320,4.8365,3.6136,...,4.3413,5.4621,4.0918,5.1019,4.9127,4.7356,5.6209,5.6611,5.5611,4.6843
ENSG00000278814.1,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658


In [30]:
# Read in the clinical saved from https://toil.xenahubs.net/download/TcgaTargetGTEX_phenotype.txt
Y = pd.read_table("/data/scratch/rcurrie/xena/TcgaTargetGTEX_phenotype.txt",
                  header=0, names=["detailed_category", "primary_site", "sample_type", "gender", "study"],
                  sep="\t", encoding="ISO-8859-1", index_col=0, dtype="str").sort_index(axis=0)

# Add column for tumor/normal
Y["tumor_normal"] = Y.apply(
    lambda row: "Normal" if row["sample_type"] in ["Cell Line", "Normal Tissue", "Solid Tissue Normal"]
    else "Tumor", axis=1)

Y.head()

,detailed_category,primary_site,sample_type,gender,study,tumor_normal
GTEX-1117F-0226-SM-5GZZ7,Adipose - Subcutaneous,Adipose Tissue,Normal Tissue,Female,GTEX,Normal
GTEX-1117F-0426-SM-5EGHI,Muscle - Skeletal,Muscle,Normal Tissue,Female,GTEX,Normal
GTEX-1117F-0526-SM-5EGHJ,Artery - Tibial,Blood Vessel,Normal Tissue,Female,GTEX,Normal
GTEX-1117F-0626-SM-5N9CS,Artery - Coronary,Blood Vessel,Normal Tissue,Female,GTEX,Normal
GTEX-1117F-0726-SM-5GIEN,Heart - Atrial Appendage,Heart,Normal Tissue,Female,GTEX,Normal


In [31]:
Y.describe()

,detailed_category,primary_site,sample_type,gender,study,tumor_normal
count,19130,19126,19131,18972,19131,19131
unique,93,46,17,2,3,2
top,Breast Invasive Carcinoma,Brain,Primary Tumor,Male,TCGA,Tumor
freq,1212,1846,9185,10456,10535,10531


In [32]:
# Label to use for stratified folds to make sure we have equal representation in
# our training and validation sets
label_for_classes = "primary_site"
label_to_predict = "tumor_normal"
output_file = "/data/scratch/rcurrie/tumor_normal.h5"

In [34]:
# Remove any samples where the selected label is null
Y_not_null = Y[pd.notnull(Y[label_for_classes])]
print("Found", Y_not_null.shape[0], "with non-null labels")

X_not_null = X[Y_not_null.index.values]

Found 19126 with non-null labels


In [35]:
# Make sure the label and example samples are in the same order
assert(Y_not_null.index.equals(X_not_null.columns))

In [45]:
# Convert classes into numbers
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(Y_not_null[label_for_classes].values)
classes = list(encoder.classes_)
classes = encoder.transform(Y_not_null[label_for_classes])
print("Total classes for stratification:", len(set(classes)))

Total classes for stratification: 46


In [60]:
# Convert prediction target into one-hot
# http://stackoverflow.com/questions/31947140/sklearn-labelbinarizer-returns-vector-when-there-are-2-classes
# so we get [0,1] and [1,0] instead of [0] and [1]
from sklearn.preprocessing import LabelBinarizer
import numpy as np

class OneHotLabelBinarizer(LabelBinarizer):
    def transform(self, y):
        Y = super().transform(y)
        if self.y_type_ == 'binary':
            return np.hstack((Y, 1-Y))
        else:
            return Y

    def inverse_transform(self, Y, threshold=None):
        if self.y_type_ == 'binary':
            return super().inverse_transform(Y[:, 0], threshold)
        else:
            return super().inverse_transform(Y, threshold)

encoder = OneHotLabelBinarizer()
y_one_hot = encoder.fit_transform(Y_not_null[label_to_predict])
labels = encoder.classes_
y_one_hot

array([[0, 1],
       [0, 1],
       [0, 1],
       ..., 
       [1, 0],
       [0, 1],
       [1, 0]])

In [68]:
"""
Write to an h5 file for training
X: examples as a numpy floating point array
y: one hot array to predict
classes: integer array of classes for stratification
features: string array of the features in X
labels: array of strings coresponding to the one-hot labels
"""
with h5py.File(output_file, "w") as f:
    f.create_dataset('X', X_not_null.shape, dtype='f')[:] = X_not_null.values
    f.create_dataset('y', one_hot_labels.shape, dtype='i')[:] = one_hot_labels
    f.create_dataset('classes', classes.shape, dtype='i')[:] = classes

    f.create_dataset('features', (len(X_not_null),1), 'S10', 
                     [l.encode("ascii", "ignore") for l in X_not_null.index.values])
    f.create_dataset('labels', (len(labels), 1), 'S10', 
                     [l.encode("ascii", "ignore") for l in labels])